### Imports

In [3]:
from tqdm import tqdm
from utils.utils import *

import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.18 ms (started: 2023-05-21 09:24:42 -05:00)


### Parameters

In [4]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

/home/ealvarez/Project/GNN_Filter
Group:		 ['pck1^', 2]
time: 2.18 ms (started: 2023-05-21 09:24:42 -05:00)


### Load dataset

In [5]:
# Load dataset Groups
df1 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int1"), delimiter="|")
df2 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int2"), delimiter="|")
df3 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int3"), delimiter="|")
df4 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int4"), delimiter="|")
# df5_ = pd.read_csv("{}/inputs/Edwin_proyecto2/{}.csv".format(dir, "int5"), delimiter="|")

""" df1 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities1"), delimiter="|")
df2 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities2"), delimiter="|")
df3 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities3"), delimiter="|")
df4 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities4"), delimiter="|")
df5_ = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities5"), delimiter="|") """

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
# print(df5_.shape)

(6245, 102)
(6245, 72)
(6245, 97)
(6245, 137)
time: 248 ms (started: 2023-05-21 09:24:42 -05:00)


In [6]:
# concat
# df_join_raw = pd.concat([df1.iloc[:,1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:], df5.iloc[:, 2:]], axis=1)
df_join_raw = pd.concat([df1.iloc[:, 1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:]], axis=1)
df_join_raw.set_index("ionMz", inplace=True)

print(df_join_raw.shape)
df_join_raw

(6245, 400)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1,47,508,360,675,232,59,345,346,504,...,45,817,32,307,410,716,228,187,361,952
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,83,174,366,1134,693,464,235,479,443,691,...,225,349,185,429,186,278,364,368,115,887
59.0453,1,1,51,642,493,143,11,227,160,154,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 42 ms (started: 2023-05-21 09:24:42 -05:00)


In [7]:
# get WT groud
df_wt = df_join_raw.filter(regex="WT", axis=1)

print(df_wt.shape)
df_wt

(6245, 200)


,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,528,1,378,438,530,250,662,431,305,334,...,45,817,32,307,410,716,228,187,361,952
59.0137,56523,24625,78586,76559,67449,74919,88152,99718,86001,93962,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2290,927,3149,3038,2774,3419,3911,3820,3374,3746,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,320,195,1137,642,588,799,602,606,768,372,...,225,349,185,429,186,278,364,368,115,887
59.0453,196,59,555,405,234,413,46,148,95,25,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2050,1415,3851,2285,2038,3089,2737,2788,1819,2600,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,2681,1984,3570,2169,2479,2668,3033,3271,2264,2411,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,1516,1307,2299,2363,2877,1436,1771,2326,1629,2740,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 23.3 ms (started: 2023-05-21 09:24:42 -05:00)


In [8]:
# get ZW groud
df_zw = df_join_raw.filter(like="zwf1^")

print(df_zw.shape)
df_zw

(6245, 120)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0861 / zwf1^ 1.4,0862 / zwf1^ 1.4,0863 / zwf1^ 1.4,0864 / zwf1^ 1.4,0865 / zwf1^ 1.4,0866 / zwf1^ 1.4,0867 / zwf1^ 1.4,0868 / zwf1^ 1.4,0869 / zwf1^ 1.4,0870 / zwf1^ 1.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1,47,508,360,675,232,59,345,346,504,...,344,790,851,372,899,752,54,254,145,1236
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,134377,133636,129520,122223,133150,129628,127563,116371,114443,118647
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,5207,5676,5416,5091,5594,5638,5234,4746,4308,5475
59.0370,83,174,366,1134,693,464,235,479,443,691,...,939,946,1195,700,1071,1302,993,857,412,1348
59.0453,1,1,51,642,493,143,11,227,160,154,...,225,168,398,84,76,516,127,351,1,753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1647,3001,2639,2850,2716,2415,2060,2886,2558,2891
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,2002,3333,2695,2343,2433,2471,2424,2671,2323,2457
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,2782,2553,2499,1393,2354,2002,2307,2151,2031,1425


time: 26.6 ms (started: 2023-05-21 09:24:42 -05:00)


In [9]:
# get pc groud
df_pc = df_join_raw.filter(like="pck1^")

print(df_pc.shape)
df_pc

(6245, 80)


,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,901,81,377,488,883,426,421,689,383,222,...,857,655,431,335,200,670,256,288,315,427
59.0137,33968,22565,36285,30275,34858,28558,36375,31684,25583,24293,...,59883,65860,65463,64715,52758,56120,60780,64502,62149,66849
59.0291,1947,945,2010,1776,1806,1549,2015,1494,1399,1029,...,2528,3118,2863,2896,2422,2483,2878,3035,2456,2964
59.0370,644,139,752,607,694,685,681,404,619,117,...,522,911,995,655,740,733,652,693,697,724
59.0453,139,78,352,414,373,388,376,353,345,134,...,181,459,429,155,321,257,342,135,315,415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,1169,923,815,1219,638,1161,1461,1710,1224,947,...,1580,1340,1940,1714,1266,735,828,1709,978,1051
996.7096,1122,1767,1914,1748,1575,1599,1738,1701,2425,1651,...,1330,817,1456,1113,816,520,161,1229,596,447
997.5542,1135,1849,1313,819,1883,1296,964,1236,1117,711,...,1111,1401,467,2230,1664,1296,391,587,883,762


time: 25.9 ms (started: 2023-05-21 09:24:43 -05:00)


In [10]:
# Logarithm

if group[0] == "WT": # Unable this line when it's run by group
  df_join_raw = df_wt.copy()
elif group[0] == "zwf1^":
  df_join_raw = df_zw.copy()
elif group[0] == "pck1^":
  df_join_raw = df_pc.copy()

df_raw_log = df_join_raw.copy()
for column in df_join_raw.columns:
  df_raw_log[column] = np.log10(df_join_raw[column])

df_raw_log

/home/ealvarez/miniconda3/envs/Conda_vgae_3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.954725,1.908485,2.576341,2.688420,2.945961,2.629410,2.624282,2.838219,2.583199,2.346353,...,2.932981,2.816241,2.634477,2.525045,2.301030,2.826075,2.408240,2.459392,2.498311,2.630428
59.0137,4.531070,4.353435,4.559727,4.481084,4.542302,4.455728,4.560803,4.500840,4.407951,4.385481,...,4.777304,4.818622,4.815996,4.811005,4.722288,4.749118,4.783761,4.809573,4.793434,4.825095
59.0291,3.289366,2.975432,3.303196,3.249443,3.256718,3.190051,3.304275,3.174351,3.145818,3.012415,...,3.402777,3.493876,3.456821,3.461799,3.384174,3.394977,3.459091,3.482159,3.390228,3.471878
59.0370,2.808886,2.143015,2.876218,2.783189,2.841359,2.835691,2.833147,2.606381,2.791691,2.068186,...,2.717671,2.959518,2.997823,2.816241,2.869232,2.865104,2.814248,2.840733,2.843233,2.859739
59.0453,2.143015,1.892095,2.546543,2.617000,2.571709,2.588832,2.575188,2.547775,2.537819,2.127105,...,2.257679,2.661813,2.632457,2.190332,2.506505,2.409933,2.534026,2.130334,2.498311,2.618048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.067815,2.965202,2.911158,3.086004,2.804821,3.064832,3.164650,3.232996,3.087781,2.976350,...,3.198657,3.127105,3.287802,3.234011,3.102434,2.866287,2.918030,3.232742,2.990339,3.021603
996.7096,3.049993,3.247237,3.281942,3.242541,3.197281,3.203848,3.240050,3.230704,3.384712,3.217747,...,3.123852,2.912222,3.163161,3.046495,2.911690,2.716003,2.206826,3.089552,2.775246,2.650308
997.5542,3.054996,3.266937,3.118265,2.913284,3.274850,3.112605,2.984077,3.092018,3.048053,2.851870,...,3.045714,3.146438,2.669317,3.348305,3.221153,3.112605,2.592177,2.768638,2.945961,2.881955


time: 57.5 ms (started: 2023-05-21 09:24:43 -05:00)


In [11]:
subgroups = [item.split("{} ".format(group[0]))[1].split(".")[0] for item in list(df_raw_log.columns)]
subgroups = np.unique(subgroups)
subgroups

array(['1', '2'], dtype='<U1')

time: 4.34 ms (started: 2023-05-21 09:24:43 -05:00)


In [12]:
# Split graph

list_raw = []

for item in subgroups:
  list_raw.append(df_raw_log.filter(like="{} {}.".format(group[0], item)))

# list_raw = [df_raw_log] # only for create one graph
print(len(list_raw))
list_raw[0]

2


,0031 / pck1^ 1.2,0032 / pck1^ 1.2,0033 / pck1^ 1.2,0034 / pck1^ 1.2,0035 / pck1^ 1.2,0036 / pck1^ 1.2,0037 / pck1^ 1.2,0038 / pck1^ 1.2,0039 / pck1^ 1.2,0040 / pck1^ 1.2,...,0761 / pck1^ 1.4,0762 / pck1^ 1.4,0763 / pck1^ 1.4,0764 / pck1^ 1.4,0765 / pck1^ 1.4,0766 / pck1^ 1.4,0767 / pck1^ 1.4,0768 / pck1^ 1.4,0769 / pck1^ 1.4,0770 / pck1^ 1.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1.431364,2.714330,2.816904,2.983175,2.883661,2.522444,2.567026,2.740363,1.255273,2.117271,...,2.380211,2.793092,2.437751,2.805501,2.209515,2.781755,2.667453,2.770115,2.665581,2.875640
59.0137,4.882285,5.211590,5.134623,4.878763,4.807887,4.708574,4.812947,4.655590,4.749257,4.803935,...,4.951716,5.058365,4.994445,5.073960,4.979175,4.760287,5.024321,4.936061,4.972966,4.963623
59.0291,3.424065,3.835944,3.729570,3.467460,3.447778,3.352954,3.410777,3.342817,3.364926,3.462997,...,3.526856,3.674402,3.605628,3.670431,3.622525,3.413467,3.617000,3.592621,3.638190,3.554004
59.0370,2.532754,3.143951,3.039414,2.536558,2.663701,2.531479,2.485721,2.613842,2.603144,2.931966,...,2.702431,2.945961,2.961421,2.982723,2.943000,2.868644,2.817565,2.927370,3.071514,2.972666
59.0453,0.000000,2.068186,2.451786,2.382017,2.214844,2.075547,0.000000,2.214844,1.255273,2.570543,...,2.363612,2.107210,2.382017,2.600973,2.686636,2.618048,2.113943,2.532754,2.832509,2.576341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.597037,3.485153,3.498448,3.436799,3.415641,3.426023,3.421275,3.341632,3.264109,3.473049,...,3.490239,3.517724,3.411956,3.400020,3.500511,3.272538,3.157457,3.398114,3.433130,3.349472
996.7096,3.311542,3.200577,3.312177,3.319522,3.219323,3.192010,3.172895,3.362482,2.975432,3.044932,...,3.289589,3.515211,3.264818,3.413300,3.303412,3.027350,3.103462,3.338656,3.374198,3.148294
997.5542,3.368845,3.277380,3.350442,3.549249,3.355452,3.373647,3.426186,3.210586,3.504607,3.536306,...,3.325721,3.221414,3.376394,3.353916,-inf,-inf,3.432649,3.400883,3.451633,3.360972


time: 35.7 ms (started: 2023-05-21 09:24:43 -05:00)


In [13]:
# Transpose

list_raw_t = []
for k in range(len(list_raw)):
  list_raw_t.append(transpose(list_raw[k]))

print(len(list_raw_t))
list_raw_t[0]

2


ionMz,59.0049,59.0137,59.0291,59.0370,59.0453,60.0171,60.9871,61.9884,71.0139,74.0249,...,992.8229,993.5530,994.9357,995.4648,995.9381,996.5509,996.7096,997.5542,997.7131,998.4845
0,1.431364,4.882285,3.424065,2.532754,0.000000,3.107549,2.127105,3.344196,3.148294,3.688242,...,0.000000,3.579784,3.464490,3.213783,3.078457,3.597037,3.311542,3.368845,2.927883,3.498448
1,2.714330,5.211590,3.835944,3.143951,2.068186,3.555094,2.807535,3.639785,3.652440,4.195983,...,2.760422,3.489818,3.588720,3.382197,3.181272,3.485153,3.200577,3.277380,3.059185,3.494155
2,2.816904,5.134623,3.729570,3.039414,2.451786,3.593397,3.384174,3.691965,3.644931,4.142327,...,1.785330,3.456366,3.672283,3.269513,3.210051,3.498448,3.312177,3.350442,3.161967,3.553033
3,2.983175,4.878763,3.467460,2.536558,2.382017,3.109241,2.928396,3.319522,3.445137,3.801541,...,2.269513,3.432167,3.627263,3.279895,3.087426,3.436799,3.319522,3.549249,3.096910,3.370513
4,2.883661,4.807887,3.447778,2.663701,2.214844,3.407391,2.767898,3.355643,3.230193,3.294466,...,2.348305,3.508260,3.578181,3.442793,2.238046,3.415641,3.219323,3.355452,3.030195,3.389875
5,2.522444,4.708574,3.352954,2.531479,2.075547,3.006038,2.666518,3.159868,3.129690,3.511482,...,2.190332,3.412461,3.492760,3.273233,3.046105,3.426023,3.192010,3.373647,2.770115,3.374748
6,2.567026,4.812947,3.410777,2.485721,0.000000,3.329601,3.095518,3.203305,3.113943,3.683587,...,2.768638,3.426836,3.542078,3.234770,3.084576,3.421275,3.172895,3.426186,3.157759,3.316390
7,2.740363,4.655590,3.342817,2.613842,2.214844,3.375298,2.770852,3.239299,3.094122,3.678427,...,3.197005,3.309204,3.558589,3.168497,3.176959,3.341632,3.362482,3.210586,2.749736,3.407391
8,1.255273,4.749257,3.364926,2.603144,1.255273,3.173769,2.212188,3.284882,3.165244,3.409595,...,2.814248,3.215373,3.487704,3.330008,3.022428,3.264109,2.975432,3.504607,3.150142,3.367356
9,2.117271,4.803935,3.462997,2.931966,2.570543,3.086360,2.403121,3.396199,3.026533,3.647187,...,2.414973,3.349278,3.562887,3.220631,3.138934,3.473049,3.044932,3.536306,3.076640,3.281033


time: 46.9 ms (started: 2023-05-21 09:24:43 -05:00)


In [14]:
# Correlation matrix

list_raw_corr = []
for df in tqdm(list_raw_t):
  matrix = df.corr(method="pearson") # pearson, kendall, spearman
  list_raw_corr.append(matrix)

print(len(list_raw_corr))
list_raw_corr[0]

 50%|██████████████████████▌                      | 1/2 [00:05<00:05,  5.51s/it]


KeyboardInterrupt: 

time: 5.65 s (started: 2023-05-21 09:24:43 -05:00)


In [ ]:
# Build graph_

threshold = 0.5
i = 1

for matrix in tqdm(list_raw_corr):
  edges = build_graph_weight(matrix, threshold)

  # save
  weighted_edges = pd.DataFrame(edges, columns=["source", "target", "weight"])
  weighted_edges.to_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], i), index=False)
  i += 1
  print(len(edges))

print(i - 1)
weighted_edges

 50%|██████████████████████                      | 1/2 [02:18<02:18, 138.54s/it]

1629510


100%|████████████████████████████████████████████| 2/2 [04:54<00:00, 147.14s/it]

4417355
2


,source,target,weight
0,59.0049,338.1873,0.507709
1,59.0049,357.2834,0.516811
2,59.0049,453.9675,0.507832
3,59.0049,587.1823,0.521553
4,59.0049,613.3933,0.516046
...,...,...,...
4417350,996.5509,997.5542,0.618624
4417351,996.5509,998.4845,0.678071
4417352,996.7096,997.5542,0.566913
4417353,997.5542,997.7131,0.589712


time: 4min 54s (started: 2023-05-19 10:27:17 -05:00)


In [ ]:
# Create dataset - nodes/edge data
for item in tqdm(subgroups):
    weighted_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], item))
    G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr="weight")
    mapping = dict(zip(list(G.nodes()), range(G.number_of_nodes())))
    G = nx.relabel_nodes(G, mapping)
    degree = dict(G.degree())

    print("Subgroup: {}".format(item))
    graph_detail(G)

    df_nodes = pd.DataFrame(degree.items(), columns=["idx", "degree"])
    df_nodes["ionMz"] = list(mapping.keys())
    df_nodes.to_csv("{}/output_preprocessing/graph_data/{}_nodes_data_{}.csv".format(dir, group[0], item), index=False)

    edges = list(G.edges())
    df_edges = pd.DataFrame(edges, columns=["source", "target"])
    df_edges["weight"] = [G.get_edge_data(edge[0], edge[1])["weight"] for edge in edges]
    df_edges.to_csv("{}/output_preprocessing/graph_data/{}_edges_data_{}.csv".format(dir, group[0], item), index=False)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Subgroup: 1
Num. nodes: 6235
Num. edges: 1629510



 50%|██████████████████████▌                      | 1/2 [00:09<00:09,  9.96s/it]

Subgroup: 2
Num. nodes: 6234
Num. edges: 4417355



100%|█████████████████████████████████████████████| 2/2 [00:36<00:00, 18.07s/it]

time: 36.1 s (started: 2023-05-19 10:32:11 -05:00)
